**Installing Required Libraries**

In [ ]:
from IPython.display import clear_output

!pip install transformers
!pip install sentence_transformers
!pip install langchain==0.0.165
!pip install llama_index==0.6.0
!pip install git+https://github.com/openai/whisper.git -q
!pip install ffmpeg-python
!git clone https://github.com/petewarden/openai-whisper-webapp
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install torchmetrics
clear_output()

**Importing required Libraries**

In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper,GPTKeywordTableIndex
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline
import pandas as pd
import whisper
from llama_index import Document



**Defining Custom LLM Class**

In [ ]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=19999)[0]["generated_text"]
    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())

**Embedding From HuggingFace**

In [ ]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

**Extracting Text From Document**

In [ ]:
with open("/content/KnowledgeDocument(pan_card_services).txt",encoding="utf-8") as f:
    text_extracted = f.read()


In [ ]:
text_extracted=text_extracted.replace('\n',', ')
text_extracted=text_extracted.replace('\xa0',', ')
text_extracted=text_extracted.replace(' and ',',')
text_extracted=text_extracted.replace('-','and')

In [ ]:
print(text_extracted)

# About Pan Card, , ### What is Pan card?, , The PAN card is a unique tenanddigit alphanumeric identification number that is issued by the Income Tax Department of India to track the taxandrelated transactions of individuals,entities. The PAN card is mandatory for any financial transaction in India, including opening a bank account, buying or selling property,,filing income tax returns., , ### Who needs a Pan card?, , All individuals/nonandindividuals (including foreign citizens/entities) earning taxable income in India, must have a PAN card., , ### Types of PAN cards, , In India, two types of PAN cards are available: eandPAN card,physical PAN card., , 1. eandPAN card: An eandPAN card is a digitallyandsigned PAN card issued in electronic format. It contains the same PAN details as a physical PAN card but is available in a digital format. It can be downloaded online,used as a valid identification document for various purposes. The eandPAN card is usually issued in a PDF format., 2. Phys

**Creating Embedding and Stroring In Vector**

In [ ]:

text_list = [text_extracted]

documents = [Document(text=t) for t in text_list]

In [ ]:
max_input_size = 4096
num_output = 1000
max_chunk_overlap = 20
chunk_size_limit = 600
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

In [ ]:

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model,prompt_helper=prompt_helper)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


**Querying**

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("How long does the process for reprinting pan card take")
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (2225 > 512). Running this sequence through the model will result in indexing errors


The process can take around 2and3 weeks to complete


**Evaluation Matrix**


*Rouge Score: ROUGE scores evaluate the quality of the generated text based on the overlap between the generated text and the reference text.*

In [ ]:
df=pd.read_csv('/content/SampleQuestions.xlsx - Sheet1.csv')
df=df.head(4)

In [ ]:
question_list=list(df['Question'])

In [ ]:
ideal_answers_list=list(df['Ideal Answer'])

In [ ]:
answer_list=[]
for i in question_list:
  response = query_engine.query(i)
  answer_list.append(response.response)


In [ ]:
df['Bot_Given_Answers']=answer_list

In [ ]:
rouge_array=[]

In [ ]:
from torchmetrics.text.rouge import ROUGEScore
rouge = ROUGEScore()
for i in range(len(answer_list)):
  rouge_array.append(rouge(answer_list[i],ideal_answers_list[i]))

In [ ]:
df['Rouge_Score']=rouge_array

In [ ]:
rouge_array

[{'rouge1_fmeasure': tensor(0.2979),
  'rouge1_precision': tensor(0.8750),
  'rouge1_recall': tensor(0.1795),
  'rouge2_fmeasure': tensor(0.2391),
  'rouge2_precision': tensor(0.7333),
  'rouge2_recall': tensor(0.1429),
  'rougeL_fmeasure': tensor(0.2979),
  'rougeL_precision': tensor(0.8750),
  'rougeL_recall': tensor(0.1795),
  'rougeLsum_fmeasure': tensor(0.2979),
  'rougeLsum_precision': tensor(0.8750),
  'rougeLsum_recall': tensor(0.1795)},
 {'rouge1_fmeasure': tensor(0.6122),
  'rouge1_precision': tensor(0.6977),
  'rouge1_recall': tensor(0.5455),
  'rouge2_fmeasure': tensor(0.4583),
  'rouge2_precision': tensor(0.5238),
  'rouge2_recall': tensor(0.4074),
  'rougeL_fmeasure': tensor(0.5714),
  'rougeL_precision': tensor(0.6512),
  'rougeL_recall': tensor(0.5091),
  'rougeLsum_fmeasure': tensor(0.5918),
  'rougeLsum_precision': tensor(0.6744),
  'rougeLsum_recall': tensor(0.5273)},
 {'rouge1_fmeasure': tensor(0.0370),
  'rouge1_precision': tensor(1.),
  'rouge1_recall': tensor(0.0

**Using Speech/Audio queries**

In [ ]:

model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 47.2MiB/s]


In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

Passing the Input Audio and Getting the transcription. Further Querying the Vector Database

In [ ]:
audio_text = transcribe("/content/Audio Clip (2023-07-06 15_53_14).m4a")
audio_text

Detected language: en


'How long does the process for reprinting pan card take?'

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(audio_text)
print(response)

2and3 weeks
